In [1]:
!pip install catboost
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.8 MB/s eta 0:00:00


In [2]:
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
import lightgbm as lgb # Define CatBoost model
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
train = pd.read_csv('train.csv')

In [4]:
train.shape

(23738, 37)

In [5]:
x = train.isnull().sum().reset_index(name = 'count')
# cols_remove = list(x[x['count'] >= 4000]['index'])
# cols_remove =
cols_remove = ['feature_15', 'feature_29']

In [6]:
train = train.drop(columns = cols_remove)
train.shape

(23738, 35)

In [19]:
test = pd.read_csv('test.csv')
print(test.shape)
test = test.drop(columns = cols_remove)

(8902, 36)


In [21]:
test = test.drop(columns = ['id'])
test.shape

(8902, 33)

In [7]:
x = train.drop(columns = ['id','level'])
y = train['level']

In [8]:
categorical_features = x.select_dtypes(include=['object']).columns.tolist()
for col in categorical_features:
    x[col] = x[col].fillna('Others')

In [9]:
from category_encoders import TargetEncoder
from sklearn.impute import KNNImputer
numerical_features = x.select_dtypes(include=['number']).columns.tolist()
# numerical_features.remove('level')  # Exclude 'level' from numerical features
# numerical_features.remove('id')
categorical_features = x.select_dtypes(include=['object']).columns.tolist()

# Create preprocessing pipelines for numerical and categorical features
numerical_pipeline = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),
])

categorical_pipeline = Pipeline([
    # ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('target_encoder', TargetEncoder(handle_unknown='ignore')),  # Using TargetEncoder
])

# Combine the pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
    ])

# Create a pipeline with the preprocessor and your model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('classifier', YourClassifier()),  # Add your classifier here
])
x_train, x_test, y_train, y_test = train_test_split(x, y,  # Pass the DataFrame 'x' here
                                                    test_size=0.2,
                                                    random_state=43,
                                                    stratify=y)
cols = x_train.columns
# Fit the pipeline to the training data
pipeline.fit(x, y)  # Now 'x_train' is a DataFrame

# Transform the data
x_train = pipeline.transform(x_train)  # These will now be NumPy arrays
x_test = pipeline.transform(x_test)

In [10]:
x = pipeline.transform(x)

In [11]:
cb = CatBoostClassifier(n_estimators=4000,
                         one_hot_max_size=2,
                         loss_function='MultiClass',
                         eval_metric='WKappa',
                         early_stopping_rounds=10,
                         task_type='CPU',
                         random_seed=5,
                         silent=True
                        )

In [12]:
cb.fit(x_train, y_train)

In [13]:
# Make predictions
y_pred = cb.predict(x_test)

# Calculate and print the F1 score
current_f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {current_f1_score}')

F1 Score: 0.7000649195470586


In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from catboost import CatBoostClassifier
from sklearn.metrics import cohen_kappa_score, f1_score

# Define the search space
space = {
    'n_estimators': hp.quniform('n_estimators', 1000, 5000, 1000),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'depth': hp.quniform('depth', 4, 10, 1),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
    'border_count': hp.quniform('border_count', 32, 255, 1),
}

# Define the objective function
def objective(params):
    params['loss_function'] = 'MultiClass'
    params['eval_metric'] = 'WKappa'
    params['random_seed'] = 42
    params['task_type'] = 'CPU'
    params['verbose'] = False

    model = CatBoostClassifier(**params)
    model.fit(x_train, y_train)  # Fit on training data
    y_pred = model.predict(x_test)  # Predict on test data
    score = f1_score(y_test, y_pred, average='weighted')  # Calculate weighted Kappa

    return {'loss': -score, 'status': STATUS_OK}  # Minimize negative Kappa

# Run Hyperopt
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

# Print the best hyperparameters
print("Best hyperparameters:", best)

# Train the final model with the best hyperparameters
best_params = {k: int(v) if type(v) == float and v.is_equal(int(v)) else v for k, v in best.items()}
print(best_params)
best_model = CatBoostClassifier(**best_params, loss_function='MultiClass', eval_metric='WKappa', random_seed=42, task_type='CPU', verbose=False)
best_model.fit(x_train, y_train)  # Fit the final model on training data

100%|██████████| 20/20 [29:25<00:00, 88.28s/trial, best loss: -0.7065286968801414]
Best hyperparameters: {'border_count': 47.0, 'depth': 6.0, 'l2_leaf_reg': 4.715289347491913, 'learning_rate': 0.0186554539121335, 'n_estimators': 2000.0}
{'border_count': 47.0, 'depth': 6.0, 'l2_leaf_reg': 4.715289347491913, 'learning_rate': 0.0186554539121335, 'n_estimators': 2000.0}


In [17]:
y_pred = best_model.predict(x_test)

# Calculate and print the F1 score
current_f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {current_f1_score}')

F1 Score: 0.7065286968801414


In [22]:
categorical_features = test.select_dtypes(include=['object']).columns.tolist()
for col in categorical_features:
    test[col] = test[col].fillna('Others')

In [24]:
test = pipeline.transform(test)
len(test)

8902

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 2. Build OOF Predictions and 3. Generate Test Predictions
oof_predictions = np.zeros(len(x))
test_predictions = []  # List to store predictions for each fold

In [26]:
for fold, (train_index, val_index) in enumerate(skf.split(x, y)):
    print(f"Fold {fold + 1}/{n_splits}")
    X_train, X_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Initialize and train your model
    model = CatBoostClassifier(**best_params, loss_function='MultiClass', eval_metric='WKappa', random_seed=42, task_type='CPU', verbose=False)
    model.fit(X_train, y_train)

    # Predict on validation set for OOF predictions
    oof_predictions[val_index] = model.predict(X_val).reshape(-1)

    # Predict on test set and store predictions for this fold
    test_predictions.append(model.predict_proba(test))

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [27]:
cv_score = f1_score(y, oof_predictions, average='weighted')
print(f"Cross-Validation F1 Score: {cv_score}")

Cross-Validation F1 Score: 0.7012691524660216


In [28]:
final_test_predictions = np.mean(test_predictions, axis = 0)

In [29]:
final_test_class = np.argmax(final_test_predictions, axis=1)

In [30]:
len(final_test_class)

8902

In [31]:
submission = pd.read_csv('sample_submission.csv')

In [32]:
submission['level'] = final_test_class

In [33]:
submission['level'].value_counts(normalize = True)

,proportion
level,
0,0.539542
1,0.265109
3,0.185689
2,0.009661


In [34]:
submission.to_csv('submission.csv', index = False)